# Lab 4
# CS 391 E1: Foundations of Data Science, Spring 2021
### Dr. Dora Erdos (Instructor), Hannah Catabia (Teaching Fellow)

Numpy tutorial:  https://numpy.org/doc/stable/user/quickstart.html

# SVD Step-by-Step
### Originally written by Vasily Ilin and Andrew Wood, adapted to the Iris dataset by Hannah Catabia

This is a (hopefully) detailed guide of doing Singular Value Decomposition.  Before you start, you may find it helpful to review how to use the NumPy library:  https://numpy.org/doc/stable/user/quickstart.html

In [2]:
# Singular-value decomposition
import numpy as np
from numpy import array
from numpy import dot
import matplotlib.pyplot as plt
import pandas as pd

In last week's lab, we used the famous iris dataset:

In [14]:
iris = pd.read_csv('https://raw.githubusercontent.com/catabia/cs391_spring21/main/iris.csv')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


We trained a Gaussian Mixture Model to predict the groups of the three different species of iris using four columns of numerical data (sepal length, sepal width, petal length, and petal width).  It worked really well.  In fact, it only made 5 misclassifications:

In [4]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components = 3)

X = iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

gmm.fit(X)

labels = gmm.predict(X)

iris['label'] = labels

# the following line makes Jupyter display the entire DataFrame, without any ...s
pd.set_option("display.max_rows", None, "display.max_columns", None)

print(iris[['species', 'label']])

        species  label
0        setosa      1
1        setosa      1
2        setosa      1
3        setosa      1
4        setosa      1
5        setosa      1
6        setosa      1
7        setosa      1
8        setosa      1
9        setosa      1
10       setosa      1
11       setosa      1
12       setosa      1
13       setosa      1
14       setosa      1
15       setosa      1
16       setosa      1
17       setosa      1
18       setosa      1
19       setosa      1
20       setosa      1
21       setosa      1
22       setosa      1
23       setosa      1
24       setosa      1
25       setosa      1
26       setosa      1
27       setosa      1
28       setosa      1
29       setosa      1
30       setosa      1
31       setosa      1
32       setosa      1
33       setosa      1
34       setosa      1
35       setosa      1
36       setosa      1
37       setosa      1
38       setosa      1
39       setosa      1
40       setosa      1
41       setosa      1
42       se

Using more data generally leads to better performance, but lots of data can sometimes be slow and difficult to work with.  So can we reduce the number of columns we use to make our predictions?  Let's try the same analysis as above, but with one less column of data.  (I will eliminate the petal width column, but you can experiment with eliminating other columns if you like.)

In [6]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components = 3)

X = iris[['sepal_length', 'sepal_width', 'petal_length']]

gmm.fit(X)

labels = gmm.predict(X)

iris['label'] = labels

# the following line makes Jupyter display the entire DataFrame, without any ...s
pd.set_option("display.max_rows", None, "display.max_columns", None)

print(iris[['species', 'label']])

        species  label
0        setosa      0
1        setosa      0
2        setosa      0
3        setosa      0
4        setosa      0
5        setosa      0
6        setosa      0
7        setosa      0
8        setosa      0
9        setosa      0
10       setosa      0
11       setosa      0
12       setosa      0
13       setosa      0
14       setosa      0
15       setosa      0
16       setosa      0
17       setosa      0
18       setosa      0
19       setosa      0
20       setosa      0
21       setosa      0
22       setosa      0
23       setosa      0
24       setosa      0
25       setosa      0
26       setosa      0
27       setosa      0
28       setosa      0
29       setosa      0
30       setosa      0
31       setosa      0
32       setosa      0
33       setosa      0
34       setosa      0
35       setosa      0
36       setosa      0
37       setosa      0
38       setosa      0
39       setosa      0
40       setosa      0
41       setosa      0
42       se

Without the petal_width column, our GMM now makes 14 mistakes.  Yikes!  That's a lot more mistakes.  So instead of just deleting one column of data, let's use SVD to project our datapoints onto a new 3-column axis, where the 3 columns capture as much variance in the data as possible.  For an nice illustration, see the GIF on this page:  https://towardsdatascience.com/pca-and-svd-explained-with-numpy-5d13b0d2a4d8

In [15]:
# convert to a numpy array
iris = np.asarray(iris)
iris

array([[5.1, 3.5, 1.4, 0.2, 'setosa'],
       [4.9, 3.0, 1.4, 0.2, 'setosa'],
       [4.7, 3.2, 1.3, 0.2, 'setosa'],
       [4.6, 3.1, 1.5, 0.2, 'setosa'],
       [5.0, 3.6, 1.4, 0.2, 'setosa'],
       [5.4, 3.9, 1.7, 0.4, 'setosa'],
       [4.6, 3.4, 1.4, 0.3, 'setosa'],
       [5.0, 3.4, 1.5, 0.2, 'setosa'],
       [4.4, 2.9, 1.4, 0.2, 'setosa'],
       [4.9, 3.1, 1.5, 0.1, 'setosa'],
       [5.4, 3.7, 1.5, 0.2, 'setosa'],
       [4.8, 3.4, 1.6, 0.2, 'setosa'],
       [4.8, 3.0, 1.4, 0.1, 'setosa'],
       [4.3, 3.0, 1.1, 0.1, 'setosa'],
       [5.8, 4.0, 1.2, 0.2, 'setosa'],
       [5.7, 4.4, 1.5, 0.4, 'setosa'],
       [5.4, 3.9, 1.3, 0.4, 'setosa'],
       [5.1, 3.5, 1.4, 0.3, 'setosa'],
       [5.7, 3.8, 1.7, 0.3, 'setosa'],
       [5.1, 3.8, 1.5, 0.3, 'setosa'],
       [5.4, 3.4, 1.7, 0.2, 'setosa'],
       [5.1, 3.7, 1.5, 0.4, 'setosa'],
       [4.6, 3.6, 1.0, 0.2, 'setosa'],
       [5.1, 3.3, 1.7, 0.5, 'setosa'],
       [4.8, 3.4, 1.9, 0.2, 'setosa'],
       [5.0, 3.0, 1.6, 0.

Now let's normalize each column.  There are several ways to do this, but since none of our numerical data is less than 0, it's appropriate to divide each column by it's mean:

In [9]:
numerical=iris[:,:4].astype('float')

#normalize values to be between 0 and 1
B = numerical/numerical.max(axis=0)
B

array([[0.64556962, 0.79545455, 0.20289855, 0.08      ],
       [0.62025316, 0.68181818, 0.20289855, 0.08      ],
       [0.59493671, 0.72727273, 0.1884058 , 0.08      ],
       [0.58227848, 0.70454545, 0.2173913 , 0.08      ],
       [0.63291139, 0.81818182, 0.20289855, 0.08      ],
       [0.6835443 , 0.88636364, 0.24637681, 0.16      ],
       [0.58227848, 0.77272727, 0.20289855, 0.12      ],
       [0.63291139, 0.77272727, 0.2173913 , 0.08      ],
       [0.55696203, 0.65909091, 0.20289855, 0.08      ],
       [0.62025316, 0.70454545, 0.2173913 , 0.04      ],
       [0.6835443 , 0.84090909, 0.2173913 , 0.08      ],
       [0.60759494, 0.77272727, 0.23188406, 0.08      ],
       [0.60759494, 0.68181818, 0.20289855, 0.04      ],
       [0.5443038 , 0.68181818, 0.15942029, 0.04      ],
       [0.73417722, 0.90909091, 0.17391304, 0.08      ],
       [0.72151899, 1.        , 0.2173913 , 0.16      ],
       [0.6835443 , 0.88636364, 0.1884058 , 0.16      ],
       [0.64556962, 0.79545455,

We can use the SVD method from SciPy or NumPy to calculate $U$, $\Sigma$, and $V^T$:

In [10]:
from scipy.linalg import svd

U,S,VT = svd(B)

print(S.shape)
print(S)

(4,)
[15.64002876  3.81901467  0.83849472  0.36912544]


Note that $\Sigma$ (above saved as the variable **S**) is not a 4 by 150 matrix, but instead just a list of the four singular values.  We can't multiply $U \times \Sigma \times V^T$ until we fix this because the dimensions won't match up:

In [155]:
Sigma = np.zeros((B.shape[0], B.shape[1]))
# populate Sigma with n x n diagonal matrix

S = np.diag(S)
print(S.shape)
z = np.zeros([B.shape[0] - S.shape[0], S.shape[1] ])
z.shape
Sigma = np.concatenate((S, z), axis =0)
print(Sigma)

(4, 4)
[[15.64002876  0.          0.          0.        ]
 [ 0.          3.81901467  0.          0.        ]
 [ 0.          0.          0.83849472  0.        ]
 [ 0.          0.          0.          0.36912544]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.       

Now $\Sigma$ is a 4 by 150 matrix. So let's check that $U \times \Sigma \times V^T$ is indeed equal B (it better be).

In [156]:
print('U x Sigma x VT:')
print(U.dot(Sigma.dot(VT)))

print()
print('B:')
print(B)

print()

print('U x Sigma x VT - B:')
print(U.dot(Sigma.dot(VT)) -B)

U x Sigma x VT:
[[0.64556962 0.79545455 0.20289855 0.08      ]
 [0.62025316 0.68181818 0.20289855 0.08      ]
 [0.59493671 0.72727273 0.1884058  0.08      ]
 [0.58227848 0.70454545 0.2173913  0.08      ]
 [0.63291139 0.81818182 0.20289855 0.08      ]
 [0.6835443  0.88636364 0.24637681 0.16      ]
 [0.58227848 0.77272727 0.20289855 0.12      ]
 [0.63291139 0.77272727 0.2173913  0.08      ]
 [0.55696203 0.65909091 0.20289855 0.08      ]
 [0.62025316 0.70454545 0.2173913  0.04      ]
 [0.6835443  0.84090909 0.2173913  0.08      ]
 [0.60759494 0.77272727 0.23188406 0.08      ]
 [0.60759494 0.68181818 0.20289855 0.04      ]
 [0.5443038  0.68181818 0.15942029 0.04      ]
 [0.73417722 0.90909091 0.17391304 0.08      ]
 [0.72151899 1.         0.2173913  0.16      ]
 [0.6835443  0.88636364 0.1884058  0.16      ]
 [0.64556962 0.79545455 0.20289855 0.12      ]
 [0.72151899 0.86363636 0.24637681 0.12      ]
 [0.64556962 0.86363636 0.2173913  0.12      ]
 [0.6835443  0.77272727 0.24637681 0.08     

Indeed, they are equal (though there are some insignificant floating-point errors).

Now let's project vectors (septal length, sepal width, petal length, petal width) from the original four-dimensional space to a three-dimensional space.

In [11]:
# Take the transpose of VT to get V

V = VT.T
reduced_dimensions = V[:,:3]
#print(V)
#print(reduced_dimensions)
#print(reduced_dimensions.shape)
# take the first 2 dimensions
projected = B.dot(reduced_dimensions)

print(projected.shape)
print(projected)

(150, 3)
[[-9.21222603e-01  4.98346043e-01 -7.85804919e-03]
 [-8.46585160e-01  4.22560431e-01  3.15722700e-02]
 [-8.49181339e-01  4.51177202e-01 -7.71112037e-03]
 [-8.43120067e-01  4.21920744e-01  9.94556287e-03]
 [-9.25841716e-01  5.09552844e-01 -2.41558337e-02]
 [-1.04450328e+00  4.94605122e-01 -5.87121943e-02]
 [-8.89092838e-01  4.45354181e-01 -5.15030334e-02]
 [-9.08519285e-01  4.75292755e-01  3.11081276e-03]
 [-7.97787293e-01  3.94423679e-01  1.18198229e-02]
 [-8.48541343e-01  4.55240831e-01  5.18632158e-02]
 [-9.73918503e-01  5.28664767e-01 -3.72393734e-03]
 [-9.00435080e-01  4.63446268e-01 -2.21810982e-03]
 [-8.22553933e-01  4.44593884e-01  4.94564352e-02]
 [-7.65812166e-01  4.49095102e-01 -6.48888300e-04]
 [-1.01939158e+00  6.00646289e-01 -3.06221513e-02]
 [-1.11321975e+00  5.85618733e-01 -1.05509784e-01]
 [-1.01793509e+00  5.19417803e-01 -8.54634794e-02]
 [-9.37890705e-01  4.73490932e-01 -3.17505863e-02]
 [-1.03794231e+00  5.13896749e-01 -6.50512873e-03]
 [-9.80479466e-01  5.0

Now, if we train our GMM on these three columns of data, how good are the predictions?

In [16]:
from sklearn.mixture import GaussianMixture

iris = pd.read_csv('https://raw.githubusercontent.com/catabia/cs391_spring21/main/iris.csv')
iris = np.asarray(iris)

gmm = GaussianMixture(n_components = 3)

X = projected

gmm.fit(X)

labels = gmm.predict(X)
labels = np.reshape(labels, (150,1))
print(iris.shape)
print(labels.shape)

iris = np.concatenate((iris,labels), axis=1)

# the following line makes Jupyter display the entire DataFrame, without any ...s
pd.set_option("display.max_rows", None, "display.max_columns", None)
import sys
np.set_printoptions(threshold=sys.maxsize)
iris

(150, 5)
(150, 1)


array([[5.1, 3.5, 1.4, 0.2, 'setosa', 1],
       [4.9, 3.0, 1.4, 0.2, 'setosa', 1],
       [4.7, 3.2, 1.3, 0.2, 'setosa', 1],
       [4.6, 3.1, 1.5, 0.2, 'setosa', 1],
       [5.0, 3.6, 1.4, 0.2, 'setosa', 1],
       [5.4, 3.9, 1.7, 0.4, 'setosa', 1],
       [4.6, 3.4, 1.4, 0.3, 'setosa', 1],
       [5.0, 3.4, 1.5, 0.2, 'setosa', 1],
       [4.4, 2.9, 1.4, 0.2, 'setosa', 1],
       [4.9, 3.1, 1.5, 0.1, 'setosa', 1],
       [5.4, 3.7, 1.5, 0.2, 'setosa', 1],
       [4.8, 3.4, 1.6, 0.2, 'setosa', 1],
       [4.8, 3.0, 1.4, 0.1, 'setosa', 1],
       [4.3, 3.0, 1.1, 0.1, 'setosa', 1],
       [5.8, 4.0, 1.2, 0.2, 'setosa', 1],
       [5.7, 4.4, 1.5, 0.4, 'setosa', 1],
       [5.4, 3.9, 1.3, 0.4, 'setosa', 1],
       [5.1, 3.5, 1.4, 0.3, 'setosa', 1],
       [5.7, 3.8, 1.7, 0.3, 'setosa', 1],
       [5.1, 3.8, 1.5, 0.3, 'setosa', 1],
       [5.4, 3.4, 1.7, 0.2, 'setosa', 1],
       [5.1, 3.7, 1.5, 0.4, 'setosa', 1],
       [4.6, 3.6, 1.0, 0.2, 'setosa', 1],
       [5.1, 3.3, 1.7, 0.5, 'setos

Only 6 misclassifications!  That's only 1 more than our original model, which was trained on all four columns of data.  Not bad!